In [ ]:
#Google Colab で実行
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import numpy as np

!pip install sklearn

In [ ]:
#データのロード
import tensorflow as tf
#訓練データとそれ以外に分ける
(X_train, Y_train), (X_test, Y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.reshape(60000, 1, 1, 28, 28).astype('float32')
X_test = X_test.reshape(10000, 1, 1, 28, 28).astype('float32')

#標準化
X_train /= 255
X_test /= 255

Y_train = Y_train.astype('long')
Y_test = Y_test.astype('long')
 
#テスト用データと検証用データに分ける
from sklearn.model_selection import train_test_split
X_val, X_test, Y_val, Y_test = train_test_split(X_test, Y_test, test_size=0.5)

batch_size = 100
print(X_val.shape, Y_val.shape)

In [ ]:
import torch.nn.functional as F
#モデルの構築
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()

    self.conv1 = nn.Conv2d(1, 16, 4, 2, padding=1)
    self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
    self.dropout1 = nn.Dropout(0.30)
    self.dropout2 = nn.Dropout(0.60)

    self.layer1 = nn.Linear(64*14*14, 128)
    self.layer2 = nn.Linear(128, 10)

    self.relu = nn.ReLU()
    self.softplus = nn.Softplus()
    
  def forward(self, x):
    x = self.relu(self.conv1(x))
    x = self.relu(self.conv2(x))
    x = F.max_pool2d(x, (2, 2))
    x = self.dropout1(x)

    x = x.view(-1, 664*14*14)
    x = self.relu(self.layer1(x))
    x = self.dropout2(x)
    x = self.softplus(self.layer2(x))
    return x
 
model = CNN().to(device)

In [ ]:
#誤差関数、学習率、最適化手法の指定
criterion = nn.CrossEntropyLoss()
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
#訓練、検証
def train(x_train, y_train):
  model.train()
  x_train = torch.from_numpy(x_train).to(device)
  y_train = torch.from_numpy(np.array(y_train)).to(device)
  y_train = (y_train.view(1, )).long()
  optimizer.zero_grad()
  pred_y = model(x_train)
  loss = criterion(pred_y, y_train)
  loss.backward()
  optimizer.step()
  return loss.item()

def eval(x_val, y_val):
  model.eval()
  x_val = torch.from_numpy(x_val).cuda()
  y_val = torch.from_numpy(np.array(y_val)).cuda()
  pred_y = model(x_val)
  y_val = (y_val.view(1, )).long()
  loss = criterion(pred_y, y_val)
  pred = pred_y.argmax()
  if pred == y_val:
    acc = 1
  else:
    acc = 0
  return acc

In [ ]:
#epoch数の指定
EPOCHS = 200

loss_train_history = []
acc_history = []

for epoch in range(EPOCHS):
  loss_train = 0
  acc = 0

  for i in range(batch_size):
    n = np.random.randint(0, high=len(X_train))
    train_x, train_y = X_train[n], Y_train[n]
    loss_train_i = train(train_x, train_y)
    loss_train += loss_train_i

  for i in range(batch_size):
    n = np.random.randint(0, high=len(X_val))
    val_x, val_y = X_val[n], Y_val[n]
    acc_i = eval(val_x, val_y)
    acc += acc_i
  
  loss_train = loss_train / batch_size
  acc = acc / batch_size

  loss_train_history.append(loss_train)
  acc_history.append(acc)
  print("epoch: ", epoch)
  print("loss_train: ", loss_train)
  print("acc: ", acc)